#Imports

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

#My Three Layer Neural Network

In [2]:
class My_Three_Line_Neural_Network:

  def __init__(self):

    pass

  def _sigmoid_(self, n):

    return 1 / (1 + np.exp(-n))

  def _forward_(self, input, Wjis, Wkjs):

    """
    X : [1, x] : Number of neurons in the first layer + 1 -> 9 * 1
    Wjis : Number of neurons in the second layer * Number of neurons in the first layer -> 3 * 9
    Wkjs : Number of neurons in the third layer * (Number of neurons in the second layer + 1) -> 8 * 4
    """

    self.SUMjis = np.dot(Wjis, input) # SUMjis : 3 * 1
    Y = self._sigmoid_(self.SUMjis) # 3 * 1

    Y_ = np.append(1, Y) # 4 * 1

    SUMkjs = np.dot(Wkjs, Y_) # SUMkjs : 8 * 1
    Z = self._sigmoid_(SUMkjs) # 8 * 1

    return Z, Y_, Y

  def backward(self, input, Wjis, Wkjs, t, hidden_size, lr):

    """
    t : target
    """

    l_output = len(input) - 1 #ignore b

    z, y_, y = self._forward_(input, Wjis, Wkjs)
    
    delta_k = z * (1 - z) * (t - z) # 8 * 1

    delta_Wkj = [[] for i in range(l_output)]

    for d in range(len(delta_k)):

      for i in y_:

        delta_Wkj[d].append(i * delta_k[d])
    
    Wkjs_without_b = Wkjs[:, 1:] # ignore b

    delta_j = y * (1 - y) * np.dot(Wkjs_without_b.T, delta_k) # 3 * 1

    delta_Wji = [[] for i in range(hidden_size)]

    for d in range(len(delta_j)):

      for i in input:

        delta_Wji[d].append(i * delta_j[d])

    """
    delta_Wkj -> 8 * 4
    delta_Wji -> 3 * 9
    """

    delta_Wkj = np.array(delta_Wkj)
    delta_Wji = np.array(delta_Wji)

    Wkjs = Wkjs + lr * delta_Wkj
    Wjis = Wjis + lr * delta_Wji

    return Wkjs, Wjis

  def fit(self, df, hidden_size, epochs, lr):

    """
    epochs : number of iterations
    lr : learning rate
    """

    features, target = df, df

    self.Wji = np.random.random((hidden_size, len(features.columns) + 1)) / 10 # 3 * 9
    self.Wkj = np.random.random((len(features.columns), hidden_size + 1)) / 10 # 8 * 4

    for epoch in tqdm(range(epochs)):

      for idx in range(len(features)):

        new_sample = features.iloc[idx, :].to_numpy()
        new_label = target.iloc[idx, :].to_numpy()      

        X = np.append(1, new_sample)  # X = [1, x]        

        self.Wkj, self.Wji  = self.backward(X, self.Wji, self.Wkj, new_label, hidden_size, lr)

  def decimal_representation(self, df):

    self.decimal_reps = []

    for idx in range(len(df)):

      sample = df.iloc[idx, :].to_numpy()
      sample_ = np.append(1, sample)

      predicted_target, _, decimal_rep = self._forward_(sample_, self.Wji, self.Wkj)

      print(f"{sample} -> {decimal_rep}")

      self.decimal_reps.append(decimal_rep)

  def binary_representation(self, df):

    self.results = np.array(self.decimal_reps)

    self.results = np.where(self.results >= 0.5, 1, 0)

    for idx in range(len(df)):

      sample = df.iloc[idx, :].to_numpy()

      print(f"{sample} -> {self.results[idx]}")

#Loading Data

In [3]:
nb_classes = 8
targets = np.array([[i for i in range(nb_classes)]]).reshape(-1)
one_hot_targets = np.eye(nb_classes)[targets]

df = pd.DataFrame(one_hot_targets)
df.columns = [f"feature{i}" for i in range(1, nb_classes+1)]
df

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
nb_classes = 4
targets = np.array([[i for i in range(nb_classes)]]).reshape(-1)
one_hot_targets = np.eye(nb_classes)[targets]

df1 = pd.DataFrame(one_hot_targets)
df1.columns = [f"feature{i}" for i in range(1, nb_classes+1)]
df1

,feature1,feature2,feature3,feature4
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0


#Train and Test the Model -> 3 layers

In [4]:
model = My_Three_Line_Neural_Network()

In [5]:
hidden_size = 3
epochs = 100000
lr = 0.02

model.fit(df, hidden_size, epochs, lr)

100%|██████████| 100000/100000 [05:53<00:00, 282.57it/s]


In [6]:
model.decimal_representation(df)

[1. 0. 0. 0. 0. 0. 0. 0.] -> [0.00471986 0.96084039 0.20281768]
[0. 1. 0. 0. 0. 0. 0. 0.] -> [0.11281917 0.97825737 0.99385598]
[0. 0. 1. 0. 0. 0. 0. 0.] -> [0.90155709 0.02921155 0.03646487]
[0. 0. 0. 1. 0. 0. 0. 0.] -> [0.01408766 0.03509841 0.01696656]
[0. 0. 0. 0. 1. 0. 0. 0.] -> [0.86796598 0.00588594 0.97740871]
[0. 0. 0. 0. 0. 1. 0. 0.] -> [0.72544483 0.97002711 0.00572276]
[0. 0. 0. 0. 0. 0. 1. 0.] -> [0.9939108  0.97807981 0.90234121]
[0. 0. 0. 0. 0. 0. 0. 1.] -> [0.00818202 0.0371335  0.86562599]


In [7]:
model.binary_representation(df)

[1. 0. 0. 0. 0. 0. 0. 0.] -> [0 1 0]
[0. 1. 0. 0. 0. 0. 0. 0.] -> [0 1 1]
[0. 0. 1. 0. 0. 0. 0. 0.] -> [1 0 0]
[0. 0. 0. 1. 0. 0. 0. 0.] -> [0 0 0]
[0. 0. 0. 0. 1. 0. 0. 0.] -> [1 0 1]
[0. 0. 0. 0. 0. 1. 0. 0.] -> [1 1 0]
[0. 0. 0. 0. 0. 0. 1. 0.] -> [1 1 1]
[0. 0. 0. 0. 0. 0. 0. 1.] -> [0 0 1]


#Train and Test the Model -> 2 layers

In [9]:
model = My_Three_Line_Neural_Network()

In [10]:
hidden_size = 2
epochs = 50000
lr = 0.02

model.fit(df1, hidden_size, epochs, lr)

100%|██████████| 50000/50000 [01:21<00:00, 612.14it/s]


In [12]:
model.decimal_representation(df1)

[1. 0. 0. 0.] -> [0.02471676 0.02298302]
[0. 1. 0. 0.] -> [0.01943916 0.98650995]
[0. 0. 1. 0.] -> [0.98570333 0.02071661]
[0. 0. 0. 1.] -> [0.97418174 0.97486342]


In [13]:
model.binary_representation(df1)

[1. 0. 0. 0.] -> [0 0]
[0. 1. 0. 0.] -> [0 1]
[0. 0. 1. 0.] -> [1 0]
[0. 0. 0. 1.] -> [1 1]
